In [1]:
!git clone https://github.com/luongkhang04/RecSysHM.git

Cloning into 'RecSysHM'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 6), reused 23 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (24/24), 31.55 KiB | 3.15 MiB/s, done.


In [2]:
%%time
import os
import sys
import copy
from datetime import datetime
import gc
import pickle as pkl
import shelve

import pandas as pd
import numpy as np
import cudf
    
sys.path.append("/kaggle/working/")
from RecSysHM import io as h_io, sub as h_sub, cv as h_cv, fe as h_fe
from RecSysHM import modeling as h_modeling, candidates as h_can, pairs as h_pairs

CPU times: user 1.8 s, sys: 508 ms, total: 2.31 s
Wall time: 4.9 s


## Load and convert data

In [3]:
%%time

c, t, a = h_io.load_data(files=['customers.csv', 'transactions_train.csv', 'articles.csv'])        

index_to_id_dict_path = h_fe.reduce_customer_id_memory(c, [t])
t["week_number"] = h_fe.day_week_numbers(t["t_dat"])
t["t_dat"] = h_fe.day_numbers(t["t_dat"])

CPU times: user 7.04 s, sys: 2.9 s, total: 9.94 s
Wall time: 49.1 s


# Get item pairs

In [4]:
%%time

pairs_per_item = 5

week_number_pairs = {}
for week_number in [96, 97, 98, 99, 100, 101, 102, 103, 104]:
    print(f"Creating pairs for week number {week_number}")
    week_number_pairs[week_number] = h_pairs.create_pairs(
        t, week_number, pairs_per_item, verbose=False
    )

Creating pairs for week number 96
Creating pairs for week number 97
Creating pairs for week number 98
Creating pairs for week number 99
Creating pairs for week number 100
Creating pairs for week number 101
Creating pairs for week number 102
Creating pairs for week number 103
Creating pairs for week number 104
CPU times: user 46.7 s, sys: 13.2 s, total: 59.8 s
Wall time: 60 s


## Main retrieval/features function!

In [5]:
def create_candidates_with_features_df(t, c, a, customer_batch=None, **kwargs):
    # splitting cv
    features_df, label_df = h_cv.feature_label_split(
        t, kwargs["label_week"], kwargs["feature_periods"]
    )
    
    # converting relative day_number
    features_df["t_dat"] = h_fe.how_many_ago(features_df["t_dat"])
    features_df["week_number"] = h_fe.how_many_ago(features_df["week_number"])
    
    # pull out the cv week
    article_pairs_df = week_number_pairs[kwargs["label_week"]-1]
    
    # check if we can limit customers
    if len(label_df) > 0:
        customers = label_df["customer_id"].unique()
    elif customer_batch is not None:
        customers = customer_batch
    else:
        customers = None
    
    ############################################
    # creating candidates (and adding features)
    ###########################################
    
    features_db = shelve.open("features_db") 
    
    # creating candidate (and saving features created)
    recent_customer_cand, features_db["customer_article"] = (
        h_can.create_recent_customer_candidates(
            features_df,
            kwargs["ca_num_weeks"],
            customers=customers,
        )
    )
    
    (cust_last_week_cand,
     cust_last_week_pair_cand,
     features_db["clw"],
     features_db["clw_pairs"]) = h_can.create_last_customer_weeks_and_pairs(
        features_df,
        article_pairs_df,
        kwargs["clw_num_weeks"],
        kwargs["clw_num_pair_weeks"],
        customers=customers,
    )
    
    popular_cand, features_db["popular_articles"] = h_can.create_popular_article_cand(
        features_df,
        c,
        a,
        kwargs["pa_num_weeks"],
        kwargs["hier_col"],
        num_candidates=kwargs["num_recent_candidates"],
        num_articles=kwargs["num_recent_articles"],
        customers=customers,
    )
    age_bucket_can, _, _ = h_can.create_age_bucket_candidates(
        features_df,
        c,
        kwargs["num_age_buckets"],
        articles=kwargs["num_recent_articles"],
        customers=customers,
    )
    
    cand = [recent_customer_cand, cust_last_week_cand, cust_last_week_pair_cand, popular_cand ,age_bucket_can]
    cand = cudf.concat(cand).drop_duplicates()
    cand = cand.sort_values(["customer_id", "article_id"]).reset_index(drop=True)
    
    del recent_customer_cand, cust_last_week_cand, cust_last_week_pair_cand, age_bucket_can, popular_cand
    
    cand = h_can.filter_candidates(cand, t, **kwargs)
    
    # creating other features
    h_fe.create_cust_hier_features(features_df, a, kwargs["hier_cols"], features_db)
    h_fe.create_price_features(features_df, features_db)
    h_fe.create_cust_features(c, features_db)
    h_fe.create_article_cust_features(features_df, c, features_db)
    h_fe.create_lag_features(features_df, a, kwargs["lag_days"], features_db)
    h_fe.create_rebuy_features(features_df, features_db)
    h_fe.create_cust_t_features(features_df, a, features_db)
    h_fe.create_art_t_features(features_df, features_db)
    
    del features_df

    # another filter at the end, for the ones that didn't get filtered earlier
    if customers is not None:
        cand = cand[cand["customer_id"].isin(customers)]
    
    # report on recall/precision of candidates
    if kwargs["cv"]:
        ground_truth_candidates = label_df[["customer_id", "article_id"]].drop_duplicates()
        h_cv.report_candidates(cand, ground_truth_candidates)
        del ground_truth_candidates        
    
    # adding features to candidates
    cand_with_f_df = h_can.add_features_to_candidates(
        cand, features_db, c, a
    )
    
    # manually adding article features (couldn't use shelve for some reason)
    for article_col in kwargs["article_columns"]:
        art_col_map = a.set_index("article_id")[article_col]
        cand_with_f_df[article_col] = cand_with_f_df["article_id"].map(art_col_map)
    
    # limiting features
    if kwargs["selected_features"] is not None:
        cand_with_f_df = cand_with_f_df[
            ["customer_id", "article_id"] + kwargs["selected_features"]
        ]
        
    features_db.close()
    os.remove("features_db.bak"), os.remove("features_db.dir"), os.remove("features_db.dat")
    
    assert len(cand) == len(cand_with_f_df), "seem to have duplicates in the feature dfs"
    del cand
    
    return cand_with_f_df, label_df

In [6]:
def calculate_model_score(ids_df, preds, truth_df):
    predictions = h_modeling.create_predictions(ids_df, preds)
    true_labels = h_cv.ground_truth(truth_df).set_index("customer_id")["prediction"]
    score = round(h_cv.comp_average_precision(true_labels, predictions),5)
    
    return score

## Parameters - one place for all!

In [7]:
cv_params = {
    "cv": True,
    "feature_periods": 105,
    "label_week": 104,
    "index_to_id_dict_path": index_to_id_dict_path,
    "pairs_file_version": "_v3_5_ex",
    "num_recent_candidates": 36,
    "num_recent_articles": 12,
    "hier_col": "department_no",
    "ca_num_weeks": 3,
    "clw_num_weeks": 12,
    "clw_num_pair_weeks": 2,
    "pa_num_weeks": 1,
    "num_age_buckets": 4,
    "filter_recent_art_weeks": 1,
    "filter_num_articles": None,
    "lag_days": [1, 3, 14, 30],
    "article_columns": ["index_code"],
    "hier_cols": [
        "department_no", "section_no", "index_group_no", "index_code",
        "product_type_no", "product_group_name"
    ],
    "selected_features": None,
    "lgbm_params": {"n_estimators": 200, "num_leaves": 20},
    "log_evaluation": 10,
    "early_stopping": 20,
    "eval_at": 12,
    "save_model": True,
    "num_concats": 5,
}
sub_params = {
    "cv": False,
    "feature_periods": 105,
    "label_week": 105,
    "index_to_id_dict_path": index_to_id_dict_path,
    "pairs_file_version": "_v3_5_ex",
    "num_recent_candidates": 60,
    "num_recent_articles": 12,
    "hier_col": "department_no",
    "ca_num_weeks": 3,
    "clw_num_weeks": 12,
    "clw_num_pair_weeks": 2,
    "pa_num_weeks": 1,
    "num_age_buckets": 4,
    "filter_recent_art_weeks": 1,
    "filter_num_articles": None,
    "lag_days": [1, 3, 14, 30],
    "article_columns": ["index_code"],
    "hier_cols": [
        "department_no", "section_no", "index_group_no", "index_code",
        "product_type_no", "product_group_name"
    ],
    "selected_features": None,
    "lgbm_params": {
        "n_estimators": 150,
        "num_leaves": 20,    
    },
    "log_evaluation": 10,
    "eval_at": 12,
    "prediction_models": ["model_104", "model_105"],
    "save_model": True,
    "num_concats": 5,
}

In [8]:
cand_features_func = create_candidates_with_features_df
scoring_func = calculate_model_score

In [9]:
%%time
cv_weeks = [104]
results = h_modeling.run_all_cvs(
    t, c, a, cand_features_func, scoring_func, 
    cv_weeks=cv_weeks, **cv_params
)

preparing training modeling dfs for 103...
candidates recall: 8.00% (18,241/227,910)
candidates precision: 0.68% (18,241/2,693,137)
preparing training modeling dfs for 102...
candidates recall: 8.15% (19,394/238,074)
candidates precision: 0.70% (19,394/2,764,685)
preparing training modeling dfs for 101...
candidates recall: 7.83% (19,969/255,172)
candidates precision: 0.67% (19,969/2,981,683)
preparing training modeling dfs for 100...
candidates recall: 7.92% (18,277/230,825)
candidates precision: 0.65% (18,277/2,819,476)
preparing training modeling dfs for 99...
candidates recall: 7.58% (17,975/237,160)
candidates precision: 0.61% (17,975/2,924,751)
concatenating all weeks together
preparing evaluation modeling dfs...
candidates recall: 8.81% (18,820/213,728)
candidates precision: 0.76% (18,820/2,474,671)
Training until validation scores don't improve for 20 rounds
[10]	train's map@12: 0.280305	train's ndcg@12: 0.373389	validation's map@12: 0.271133	validation's ndcg@12: 0.363768
[20]

In [10]:
%%time
gc.collect()
h_modeling.full_sub_train_run(t, c, a, cand_features_func, scoring_func, **sub_params)
predictions = h_modeling.full_sub_predict_run(
    t, c, a, cand_features_func, **sub_params
)

preparing training modeling dfs for 104...
preparing training modeling dfs for 103...
preparing training modeling dfs for 102...
preparing training modeling dfs for 101...
preparing training modeling dfs for 100...
concatenating all weeks together
[10]	train's map@12: 0.275188	train's ndcg@12: 0.367375
[20]	train's map@12: 0.282115	train's ndcg@12: 0.374992
[30]	train's map@12: 0.286271	train's ndcg@12: 0.379596
[40]	train's map@12: 0.289186	train's ndcg@12: 0.38291
[50]	train's map@12: 0.292238	train's ndcg@12: 0.386305
[60]	train's map@12: 0.294584	train's ndcg@12: 0.388986
[70]	train's map@12: 0.297146	train's ndcg@12: 0.391814
[80]	train's map@12: 0.298556	train's ndcg@12: 0.393502
[90]	train's map@12: 0.300833	train's ndcg@12: 0.395759
[100]	train's map@12: 0.302333	train's ndcg@12: 0.397288
[110]	train's map@12: 0.304479	train's ndcg@12: 0.39927
[120]	train's map@12: 0.306192	train's ndcg@12: 0.40083
[130]	train's map@12: 0.307883	train's ndcg@12: 0.402494
[140]	train's map@12: 0

In [11]:
sub = h_sub.create_sub(c["customer_id"], predictions, index_to_id_dict_path)
sub.to_csv('dev_submission.csv', index=False)

display(sub.head())
print(sub.shape)

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0779781015 0568601043 0762846031 0762846027 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0918522001 0714790020 0448509014 0893432002 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0794321008 0805000001 0918522001 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0791587001 0852584001 0866731001 0861803009 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587021 0918292001 0896152001 09...


(1371980, 2)
